# FM(Factorized Machine)因子分解机

该方法常用的场景是推荐，在snippets-rec中会有FM及其衍生算法的介绍。该算法不仅仅是一个回归算法, 也可以用于分类、排序等等。

## 1 简介

### 1.1 FM视为线性模型引入特征交叉
- **特征交叉在处理多因素交互作用时是十分必要的.**
如预测购买商品的任务中, 年龄=7岁 + 女孩 = 童装连衣裙。

- 传统线性回归引入交叉项: 二阶多项式回归
$$
\hat{y}(x)=w_0 + \underbrace{\sum_{i=1}^p w_i x_i}_{\text {线性回归 }}+\underbrace{\sum_{i=1}^p \sum_{j=i+1}^p w_{i j} x_i x_j}_{\text {交叉项 (组合待征) }}
$$

其中样本$x_i$为样本x的第$i$维特征，$p$为维数.

当$X$中包含稀疏特征(如商品类别/商品标签, onehot encode后可能几万个维度)时, 有以下致命弱点:
1)交叉项会有维度爆炸;
2)参数数目极大, 过拟合;
3)当特征有稀疏性时, 交叉项也会有稀疏性而且往往更大, 这会交叉因素很难学习。
例如推荐场景下，商品作为特征是非常稀疏的，这会导致同时买过蓝牙耳机和雪地靴的人极少甚至没有. 而实际两个商品的交叉项可能是有意义的(蓝牙耳机搭配户外运动装备)

- FM引入了隐含特征, 通过矩阵分解对交叉项进行近似, 解决特征稀疏问题
因此FM通过矩阵分解对矩阵$W$进行了低秩近似.$W \approx V \cdot V^t$，其中$V \in \mathbb{R}^{p \times k}$.
1）稀疏特征场景下, $p$通常比较大, $k$往往小于$p$. FM能有效降低参数数目，防止模型泛化能力不足；

2）极端情况, 当k足够大时，FM模型等价于线性回归中引入交叉项。

$$
\begin{aligned}
交叉相部分= \\
& \sum_{i=1}^{p} \sum_{j=i+1}^p (\vec {\mathbf{v}_i} \cdot \vec {\mathbf{v}_j} ) x_i x_j \\
= & \frac{1}{2} \sum_{i=1}^p \sum_{j=1}^p (\vec {\mathbf{v}_i} \cdot \vec {\mathbf{v}_j} ) x_i x_j-\frac{1}{2} \sum_{i=1}^p (\vec {\mathbf{v}_i} \cdot \vec {\mathbf{v}_j} ) x_i x_i \\
= & \frac{1}{2}\left(\sum_{i=1}^p \sum_{j=1}^p \sum_{f=1}^k v_{i, f} v_{j, f} x_i x_j-\sum_{i=1}^p \sum_{f=1}^k v_{i, f} v_{i, f} x_i x_i\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{i=1}^{p} v_{i, f} x_i\right)\left(\sum_{j=1}^{p} v_{j, f} x_j\right)-\sum_{i=1}^{p} v_{i, f}^2 x_i^{2}\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left ( \sum_{i=1}^{p} v_{i, f} x_i \right)^{2}-\sum_{i=1}^{p} v_{i, f}^2 x_i^{2}\right)
\end{aligned}
$$

FM解决了暴力引进交叉项在稀疏特征场景下的缺点:
1)参数数目降低, 通常$k$都远小于$p$, 增加模型泛化能力;
2)通过矩阵分解来低秩近似交叉项稀疏, 保留了模型的表达能力;
3)FM还具备二阶多项式回归不具备的能力, 在交叉项极为稀疏甚至全为0的情况下学习交叉项。这其实是低秩近似的功劳。
A和i1完全没有相关数据, 在矩阵W上则表现为稀疏. 但基于低秩矩阵乘积表示则会收到其他非稀疏的交叉项影响，例如:
B、C等许多和i1有交叉项，同时有和i6有交叉项，因此FM求解结果中，v_i1和v_i6应该是非常相似，而A和i6又有交叉项, 因此A和i1的交叉项也不为0, 交叉项权重接近于A和i6.
4)FM在计算方程时的时间复杂度O(kp), 而不是二阶多项式的O(kp^2)

### 1.2 FM与因式分解
FM是对因式分解的扩展, 其表达能力比隐式分解更高.
- FM:
每一个特征都会有其隐向量, 特征交叉时, 交叉特征的权重为两特征隐向量的内积.

- 因式分解:
因式分解分解了两组变量的相关关系. 以推荐场景为例, 对用户~物品矩阵进行分解, 可以得到两组隐向量, 分别代表物品和用户.

- 因式分解可以看做FM的特殊情况
用户U和物品I都以onehot形式作为特征, 则FM可以得到一组隐向量, 包含了用户onehot特征对应的隐向量和物品onehot特征对应的隐向量, 此时能够得到等价于因式分解的表达效果.
$$
y = w_0 + w_u + w_i + \mathbb{v_u} \cdot \mathbb{v_i}
$$



下面我们将上述表达式以样本形式矩阵化，方便机器学习和深度学习引擎求解，通过下面几步阐述：
step 1)假定共有$n$个样本，即$X \in \mathbb{R}^{n \times p}$，$Y \in \mathbb{R}^{n}$
$$
X = \begin{bmatrix}
x_1^{(1)} & \dots & x_p^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
x_1^{(n)} & \dots & x_p^{(n)} \\
\end{bmatrix}
$$

step 2)$V \in \mathbb{R}^{p \times k}$
$$
V = \begin{bmatrix}
v_1^{(1)} & \dots & v_k^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
v_1^{(p)} & \dots & v_k^{(p)} \\
\end{bmatrix}
$$

step 3)Y的交叉项$Y_{inter}$

$$
Y_{inter} = \sum_{i=1}^{p} \sum_{j=i+1}^{p} \langle \textbf v_i, \textbf v_j \rangle \vec{x_i} \odot \vec{x_j}  \\
= \frac{1}{2} \sum_{f=1}^{k} \Big( \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2 - \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2 \Big) \\
= \frac{1}{2} \sum_{f=1}^{k}  \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2 -
\frac{1}{2} \sum_{f=1}^{k}  \big( \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2 \big)
$$

注意：
- 此处$\vec{x_i} $表示向量，$\vec{x_i} \in \mathbb{R}^{n \times 1}$
- $\odot$表示元素乘法(element wise product)，此处平方运算$\vec{x_i}^{2}$也表示向量的元素级平方操作.
- 记住该公式相减的一部分和第二部分, 后续会用到.

step4)Y的交叉项可以利用$XV$进一步简化:
$X$和$V$的内积($n*k$维):
$$
XV = \begin{bmatrix}
\sum_{i=1}^{p} v_f^{(1)} x_i^{(1)}  & \dots &  \sum_{i=1}^{p} v_f^{(k)} x_i^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
\sum_{i=1}^{p} v_f^{(1)} x_i^{(n)} & \dots & \sum_{i=1}^{(n)} v_f^{(k)} x_i^{(n)} \\
\end{bmatrix} =
\begin{bmatrix}
S_{1,1}^{(1)}  & \dots &  S_{1,k}^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
S_{1,1}^{(n)}  & \dots & S_{1,k}^{(n)} \\
\end{bmatrix}
$$


我们注意到:
- step3中$Y$求解第一项中求和元素$\sum_{i=1}^{p} v_f^{(i)}$和 $ \vec{x_i} $ 的乘积恰好等于$XV$的第$i$列:
$$
v_f^{(i)} \vec{x_i} = \begin{pmatrix}
  v_f^{(i)} x_i^{(1)}  \\
  ... \\
  v_f^{(i)} x_i^{(n)}
\end{pmatrix}
$$

- 因此step3中第一项$\frac{1}{2} \sum_{f=1}^{k}  \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2$等价于$XV$的元素级平方$(XV)^2 = (XV) \odot (XV)$.
我们定义一个运算表示对矩阵的列求和: $\sigma_{(1)}(B)$表示对矩阵$B$的列向量求和. 因此有:
$$
\frac{1}{2} \sum_{f=1}^{k}  \big(\sum_{i=1}^{p} v_f^{(i)} \vec{x_i} \big)^2
=\frac{1}{2} \sigma_{(1)}((XV) \odot (XV))
$$

- 同理step3中的第二项中求和元素$ \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2$可以视为$(X \odot X) \cdot (V \odot V)$的第$i$列, 第二项可以视为$(X \odot X) \cdot (V \odot V)$的列向量求和:

$$
\frac{1}{2} \sum_{f=1}^{k}  \big( \sum_{i=1}^{p}v_f^{(i) 2} \vec{x_i}^2 \big)
=\frac{1}{2} \sigma_{(1)}((X \odot X) \cdot (V \odot V))
$$

因此最终的矩阵表达式为:

$$
Y = w_0 + X w + \frac{1}{2} \sigma_{(1)}((XV) \odot (XV)) - \frac{1}{2} \sigma_{(1)}((X \odot X) \cdot (V \odot V))
$$
其中:
- $w_0$为截距项;
- $w$为线性项系数;
- $V$为交叉项的隐式特征;
- $\odot$为元素级乘法;
- $\sigma_{(1)}(B)$表示对矩阵$B$的列向量求和


In [ ]:
import torch


class TorchFM(torch.nn.Module):
    def __init__(self, n=None, k=None):
        super().__init__()
        self.V = torch.nn.Parameter(torch.randn(n, k), requires_grad=True)
        self.linear = torch.nn.Linear(n, 1)

    def forward(self, x):
        out_1 = torch.matmul(x, self.V).pow(2).sum(1, keepdim=True)  #S_1^2
        out_2 = torch.matmul(x.pow(2), self.V.pow(2)).sum(1, keepdim=True)  # S_2

        out_inter = 0.5 * (out_1 - out_2)
        out_linear = self.linear(x)
        out = out_inter + out_linear

        return out

In [2]:
import torch
vv = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [3]:
vv.pow(2)

tensor([[ 1,  4,  9],
        [16, 25, 36],
        [49, 64, 81]])

## 利用深度学习框架的Embedding层构建稀疏特征的隐向量

## 稀疏特征+稠密特征

实际应用中稠密特征往往不需要构建隐向量$V$。而稀疏特征的隐向量矩阵等价于稀疏特征的Embedding层参数。

参考: Steffen Rendle. Factorization machines. In Proceedings of the 2010 IEEE International Conference on Data Mining, pages 995–1000, 2010.
> For dense features, the high-dimensional feature vector is used directly as input, while for sparse features, the feature vector is transformed into a low-dimensional space using latent factors.


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FM(nn.Module):
    def __init__(self, num_sparse_features, num_dense_features, embedding_dim, num_hidden_units=256):
        super(FM, self).__init__()

        # 初始化稀疏特征的嵌入层
        self.sparse_embedding = nn.ModuleList([
            nn.Embedding(num_embeddings, embedding_dim) for num_embeddings in num_sparse_features
        ])

        # 初始化稠密特征的全连接层
        self.dense_layer = nn.Sequential(
            nn.Linear(num_dense_features, num_hidden_units),
            nn.BatchNorm1d(num_hidden_units),
            nn.ReLU(),
            nn.Linear(num_hidden_units, embedding_dim)
        )

        # 初始化FM交叉层
        self.linear = nn.Linear(embedding_dim * len(num_sparse_features) + num_dense_features, 1)
        self.interaction = nn.Linear(embedding_dim, embedding_dim)
        self.out = nn.Sigmoid()

    def forward(self, x_sparse, x_dense):
        # 处理稀疏特征
        sparse_embeds = [emb(x_sparse[:, i]) for i, emb in enumerate(self.sparse_embedding)]
        sparse_embeds = torch.cat(sparse_embeds, dim=1)

        # 处理稠密特征
        dense_embeds = self.dense_layer(x_dense)

        # 计算FM交叉项
        x = torch.cat([sparse_embeds, dense_embeds], dim=1)
        linear_part = self.linear(x)
        square_of_sum = torch.pow(torch.sum(x, dim=1, keepdim=True), 2)
        sum_of_square = torch.sum(x * x, dim=1, keepdim=True)
        cross_part = square_of_sum - sum_of_square
        print('cross_part:', cross_part.shape, square_of_sum.shape, sum_of_square.shape)
        print('linear_part:', linear_part.shape)
        output = linear_part + 0.5 * cross_part

        return self.out(output)

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 假设我们有一个包含稀疏特征和稠密特征的数据集，其中稀疏特征有100个，每个特征取值范围在[0, 10000]之间，
# 稠密特征有10个，每个特征的值在[0, 1]之间
sparse_features = torch.randint(0, 10000, size=(1000, 100))
dense_features = torch.rand((1000, 10))

# 假设这些数据集对应的标签是0或1
labels = torch.randint(0, 2, size=(1000,))

# 将数据集和标签包装成PyTorch的Dataset
dataset = TensorDataset(sparse_features, dense_features, labels)

# 创建DataLoader，用于批量加载数据
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 初始化FM模型
fm = FM(num_sparse_features=[10001] * 100, num_dense_features=10, embedding_dim=10)

# 定义优化器和损失函数
optimizer = torch.optim.Adam(fm.parameters())
criterion = nn.BCEWithLogitsLoss()

# 训练模型
for epoch in range(10):
    for i, (x_sparse, x_dense, y) in enumerate(dataloader):
        optimizer.zero_grad()
        y_pred = fm(x_sparse, x_dense)
        loss = criterion(y_pred.view(-1), y.float())
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f"Epoch {epoch}, batch {i}, loss {loss.item():.4f}")

cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
Epoch 0, batch 0, loss 0.7857
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([32, 1]) torch.Size([32, 1]) torch.Size([32, 1])
linear_part: torch.Size([32, 1])
cross_part: torch.Size([3

In [20]:
sparse_features_to_pred = torch.randint(0, 10000, size=(5, 100))
dense_features_to_pred = torch.rand((5, 10))
y_pred = fm(sparse_features_to_pred, dense_features_to_pred)
print(y_pred.data)

cross_part: torch.Size([5, 1]) torch.Size([5, 1]) torch.Size([5, 1])
linear_part: torch.Size([5, 1])
tensor([[1.],
        [1.],
        [0.],
        [1.],
        [1.]])
